In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('updated_data.csv')

In [3]:
#data = data.sample(frac=1,random_state=42)

In [4]:
data

,Unnamed: 0.1,Unnamed: 0,Target,Tweet
0,541200,541200,0,"['@nkluvr4eva', 'poor', 'little', 'dumple', ' ..."
1,750,750,0,"['I', 'bed', 'I', 'get', 'wake', 'hella', 'ear..."
2,766711,766711,0,"['I', 'able', 'listen', 'yet', ' ', 'speaker',..."
3,285055,285055,0,"['remember', 'solve', 'relatively', 'big', 'eq..."
4,705995,705995,0,"['eat', 'much', 'feel', 'sick']"
...,...,...,...,...
99995,159217,159217,0,"['I', 'boyfriend', 'cut', 'hair', 'instead', '..."
99996,298540,298540,0,"['funny', 'home', 'video', '.......', 'wah', '..."
99997,839945,839945,1,"['I', 'go', 'watch', 'tv', 'hopefully', 'tivo'..."
99998,732586,732586,0,"['good', 'night', 'yesterday', 'wish', 'neighb..."


In [5]:
data['Target'].value_counts()

1    50057
0    49943
Name: Target, dtype: int64

In [6]:
data = data.drop(columns=['Unnamed: 0','Unnamed: 0.1'])

In [7]:
data.columns = ['Target','Tweet']

In [8]:
data

,Target,Tweet
0,0,"['@nkluvr4eva', 'poor', 'little', 'dumple', ' ..."
1,0,"['I', 'bed', 'I', 'get', 'wake', 'hella', 'ear..."
2,0,"['I', 'able', 'listen', 'yet', ' ', 'speaker',..."
3,0,"['remember', 'solve', 'relatively', 'big', 'eq..."
4,0,"['eat', 'much', 'feel', 'sick']"
...,...,...
99995,0,"['I', 'boyfriend', 'cut', 'hair', 'instead', '..."
99996,0,"['funny', 'home', 'video', '.......', 'wah', '..."
99997,1,"['I', 'go', 'watch', 'tv', 'hopefully', 'tivo'..."
99998,0,"['good', 'night', 'yesterday', 'wish', 'neighb..."


In [9]:
condition = data['Target'] == 4

In [10]:
data.loc[condition,'Target'] = 1

In [12]:
data['Target'].unique()

array([0, 1])

## Text Preprocessing ##

In [105]:
data

,Target,Tweet
0,0,"['@nkluvr4eva', 'poor', 'little', 'dumple', ' ..."
1,0,"['I', 'bed', 'I', 'get', 'wake', 'hella', 'ear..."
2,0,"['I', 'able', 'listen', 'yet', ' ', 'speaker',..."
3,0,"['remember', 'solve', 'relatively', 'big', 'eq..."
4,0,"['eat', 'much', 'feel', 'sick']"
...,...,...
99995,0,"['I', 'boyfriend', 'cut', 'hair', 'instead', '..."
99996,0,"['funny', 'home', 'video', '.......', 'wah', '..."
99997,1,"['I', 'go', 'watch', 'tv', 'hopefully', 'tivo'..."
99998,0,"['good', 'night', 'yesterday', 'wish', 'neighb..."


In [114]:
import spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [115]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
!pip install nltk

In [106]:
import nltk

In [107]:
from nltk.corpus import stopwords

In [108]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/vishu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [109]:
stopwo = stopwords.words('english')

In [110]:
import string

In [111]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [112]:
def preprocess(data):
    #lowercase
    data = data.lower()
    data = nlp(data)
    data = [tk.lemma_ for tk in data]
    final = []
    for stp in data :
        if stp not in stopwo and stp not in string.punctuation:
            final.append(stp)

    return final

In [116]:
print(preprocess("Hello my walking went name is abcd ;+,./"))

['hello', 'walking', 'go', 'name', 'abcd']


In [22]:
from nltk.stem import PorterStemmer

In [23]:
prt = PorterStemmer()

In [39]:
def stemming(doc):
    data = doc.lower()
    data = nlp(doc)
    return [prt.stem(dd.text) for dd in data]

In [49]:
print(stemming("I am went"))

['i', 'am', 'went']


## Word2Vec ##

In [ ]:
!pip install gensim

In [89]:
data['Tweet'] = data['Tweet'].apply(preprocess)

In [ ]:
data.to_csv('updated_data.csv')

In [97]:
total = len(data['Tweet'])

In [98]:
total

100000

In [99]:
train_split = int(total * 0.7)

In [100]:
train_split

70000

In [103]:
x_train = data.iloc[:train_split,-1]
y_train = data.iloc[:train_split,0]

In [101]:
remaining_data = total - train_split

In [102]:
remaining_data

30000

In [104]:
x_test = data.iloc[train_split:train_split + 15000,-1]
y_test = data.iloc[train_split:train_split + 15000,0]

In [105]:
x_val = data.iloc[train_split + 15000:train_split + 60000,-1]
y_val = data.iloc[train_split + 15000:train_split + 60000,0]

In [7]:
import gensim
from gensim import corpora,similarities
import ast

In [8]:
vocab = [ast.literal_eval(vv) for vv in data['Tweet']]

In [ ]:
vocab

In [110]:
type(vocab[0])

list

In [9]:
model = gensim.models.Word2Vec.load("vocab.word2vec")

In [111]:
model = gensim.models.Word2Vec(window=10,min_count=2)

In [112]:
model.build_vocab(vocab)

In [114]:
model.train(vocab,total_examples=model.corpus_count,epochs=model.epochs)

(3325978, 4489625)

In [13]:
model.save("vocab.word2vec")

In [ ]:
model.wv['hate']

In [10]:
import numpy as np

In [11]:
artificial_vector = np.zeros(model.wv.vector_size)

In [12]:
artificial_vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
vectorized_data = []
index = []

for _sent in vocab:
    word_data = []
    for word in _sent:
        if word in model.wv.key_to_index:
            word_data.append(model.wv[word])
        else:
            word_data.append(artificial_vector)
    
    vectorized_data.append(word_data)

In [14]:
import torch
from torch.nn.utils.rnn import pad_sequence

/home/vishu/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
np.array(vectorized_data[8]).shape[-1]

(5, 100)

In [19]:
tensor = []
index = []
for indx,tweet in enumerate(vectorized_data):
    if np.array(tweet).ndim == 2 and np.array(tweet).shape[0] <= 20:
        tensor.append(torch.Tensor(tweet))
        index.append(indx)
    else:
        pass

/tmp/ipykernel_4070/571836077.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  tensor.append(torch.Tensor(tweet))


In [20]:
len(tensor)

99104

In [22]:
final_tensor = pad_sequence(tensor,batch_first=True)

In [23]:
final_tensor[3].shape

torch.Size([20, 100])

In [24]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [25]:
target = data.iloc[index,0]

In [26]:
total_len = final_tensor.size(0)

In [27]:
train_split = int(total_len * 0.8)

In [28]:
train_split

79283

In [30]:
remaining = total_len - train_split

In [31]:
remaining

19821

In [32]:
train_dataset = TensorDataset(final_tensor[0:train_split],torch.Tensor(target)[0:train_split])
test_dataset = TensorDataset(final_tensor[train_split:train_split + 10000],torch.Tensor(target)[train_split:train_split + 10000])
val_dataset = TensorDataset(final_tensor[train_split+10000:],torch.Tensor(target)[train_split + 10000:])

In [33]:
train_loader = DataLoader(train_dataset,batch_size=32)
test_loader = DataLoader(test_dataset,batch_size=32)
val_loader = DataLoader(val_dataset,batch_size=32)

In [185]:
!pip install lightning

In [34]:
import pytorch_lightning as pl

In [35]:
INPUT_DIM = 100
HIDDEN_DIM = 128
BATCH = 32


In [105]:
import torch.nn as nn
import torch
import torch.nn.functional as f
import torchmetrics
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping

In [154]:
class TWEET_SENTIMENT(pl.LightningModule):
    def __init__(self,input_dim,hidden_dim):
        super(TWEET_SENTIMENT,self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.criterion = nn.CrossEntropyLoss()
        self.LSTM = nn.LSTM(input_size = input_dim,hidden_size = hidden_dim,batch_first = True)
        self.h0 = torch.zeros(size = (1,32,self.hidden_dim))
        self.c0 = torch.zeros(size = (1,32,self.hidden_dim))
        self.accuracy = torchmetrics.F1Score(task="multiclass",num_classes=2)
        self.fc1 = nn.Linear(128,64)
        self.fc2 = nn.Linear(64,2)

    def forward(self,x):
        #x -> 32,38,100
        h,c = self.LSTM(x,(self.h0,self.c0))
        #h-> 32,38,128
        h = h[:,-1,:] # 32,128
        h = self.fc1(h)
        h = f.relu(h)
        h = self.fc2(h)
        h = f.softmax(h,dim=-1)
        return h

    def training_step(self,batch,batch_idx):
        x,y = batch
        if x.size(0) == 32:
            y = y.long()
            pred = self(x)
            loss = self.criterion(pred,y)
            accuracy = self.accuracy(pred,y)
            self.log_dict({'Train loss':loss , 'Accuracy':accuracy},on_epoch=True,prog_bar=True)
            return {'loss':loss}
        return None
    
    def validation_step(self,batch,batch_idx):
        x,y = batch
        if x.size(0) == 32:
            y = y.long()
            pred = self(x)
            accuracy = self.accuracy(pred,y)
            loss = self.criterion(pred,y)
            self.log_dict({'Validation loss':loss,'Accuracy':accuracy},on_epoch=True,prog_bar=True)
            return loss
        return None
    
    def test_step(self,batch,batch_idx):
        x,y = batch
        if x.size(0) == 32:
            y = y.long()
            pred = self(x)
            accuracy = self.accuracy(pred,y)
            loss = self.criterion(pred,y)
            self.log_dict({'Test loss':loss,'Accuracy':accuracy},on_epoch=True,prog_bar=True)
            return loss
        return None
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=0.001)




In [ ]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1, 
    mode='min',
)

In [101]:
trainer = pl.Trainer(accelerator='auto',
                     min_epochs=1,max_epochs=10,
                     callbacks=[EarlyStopping(monitor='val_loss',patience=3),checkpoint_callback])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [156]:
tweet_model = TWEET_SENTIMENT(INPUT_DIM,128)

In [166]:
tweet_model = torch.load('TWEET_MODEL.pt')

In [167]:
tweet_model

TWEET_SENTIMENT(
  (criterion): CrossEntropyLoss()
  (LSTM): LSTM(100, 128, batch_first=True)
  (accuracy): MulticlassF1Score()
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

In [103]:
trainer.fit(tweet_model,train_loader,val_loader)

Epoch 6:  12%|█▏        | 291/2478 [00:12<01:33, 23.31it/s, v_num=6, Train loss_step=0.403, Accuracy_step=0.812, Validation loss=0.510, Accuracy=0.748, Train loss_epoch=0.478, Accuracy_epoch=0.764] 

In [104]:
torch.save(tweet_model,"TWEET_MODEL.pt")

In [158]:
trainer.test(tweet_model,test_loader)

/home/vishu/miniconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0:   2%|▏         | 5/313 [00:00<00:31,  9.84it/s]

Testing DataLoader 0: 100%|██████████| 313/313 [00:11<00:00, 26.56it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         Accuracy          │    0.7517027258872986     │
│         Test loss         │     0.562911331653595     │
└───────────────────────────┴───────────────────────────┘

[{'Test loss': 0.562911331653595, 'Accuracy': 0.7517027258872986}]

In [124]:
trainer.validate(tweet_model,val_loader)

/home/vishu/miniconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Validation DataLoader 0: 100%|██████████| 307/307 [00:09<00:00, 32.20it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         Accuracy          │    0.7528594732284546     │
│      Validation loss      │    0.4996325373649597     │
└───────────────────────────┴───────────────────────────┘

[{'Validation loss': 0.4996325373649597, 'Accuracy': 0.7528594732284546}]

In [175]:
tweet = "My dog is missing from 9 days"
tweet = preprocess(tweet)
tweet = [model.wv[t] for t in tweet]
tweet = torch.Tensor(tweet) #5,100
tweet = tweet.unsqueeze(0) #1,5,100


In [176]:
tweet = tweet.repeat(32,1,1)

In [177]:
tweet.shape

torch.Size([32, 4, 100])

In [178]:
tweet_model.eval()
with torch.no_grad():
    predi = tweet_model(tweet)

In [179]:
result = torch.argmax(predi[0])

In [180]:
if result.item() == 1:
    print("Positive Sentiment")
else:
    print("Negative Sentiment")

Negative Sentiment
